In [ ]:
# Imports

import geopandas as gpd
from shapely.geometry import Point, box
from random import uniform
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
from shapely.geometry import shape
from shapely.ops import unary_union
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# algorithm from: https://gis.stackexchange.com/questions/364058/finding-closest-point-to-shapefile-coastline-python
# Jeremy Malczyk Jun 6 '20 at 6:34
Antartica_poly = unary_union(gpd.read_file("./Quantarctica3/Basemap/ADD_Coastline_low_res_polygon.shp").to_crs('EPSG:3031').geometry)

#single geom for Antartica
Antarctica = Antartica_poly
#print(Antarctica)

#single geom for the coastline
coastline = unary_union(gpd.read_file(".Quantarctica3/Basemap/ADD_Coastline_high_res_line_Sliced.shp").to_crs('EPSG:3031').geometry)
#print(coastline)

def make_point(id):
    point = None
    while point is None or not Antarctica.contains(point):
        point = Point(uniform(Antarctica.bounds[0],Antarctica.bounds[2]),
                      uniform(Antarctica.bounds[1],Antarctica.bounds[3]))
    return {"id": id, "geometry": point}

def compute_distance(point):
    point['dist_to_coastline'] = point['geometry'].distance(coastline)
    return point

with ThreadPoolExecutor(max_workers=4) as tpe:
    points = list(tqdm(tpe.map(make_point, range(10000)), desc="generating points", total=10000))
    result = list(tqdm(tpe.map(compute_distance, points), desc="computing distances", total=len(points)))

gdf = gpd.GeoDataFrame.from_records(result)
print(gdf)

ax = gpd.GeoDataFrame.from_records([{"geometry":coastline}]).plot(figsize=(18,18), color='black')
ax = gdf.plot(ax=ax, column='dist_to_coastline', cmap='rainbow',legend=True,legend_kwds={'shrink': 0.3})
plt.show()

In [ ]:
# Now modified

In [ ]:
# import Megadune coordinates

# Read the csv file
SA_df = pd.read_csv("./SiteA_megadune_pos.csv")
SA_df = SA_df.drop(['level_0'], axis = 1)
print("SA:", len(SA_df))

SB_df = pd.read_csv("./SiteB_megadune_pos.csv")
SB_df = SB_df.drop(['level_0'], axis = 1)
print("SB:", len(SB_df))

SC_df = pd.read_csv("./SiteC_megadune_pos.csv")
SC_df = SC_df.drop(['level_0'], axis = 1)
print("SC:", len(SC_df))

SD_df = pd.read_csv("./SiteD_megadune_pos.csv")
SD_df = SD_df.drop(['level_0'], axis = 1)
print("SD:", len(SD_df))

SE_df = pd.read_csv("./SiteE_megadune_pos.csv")
SE_df = SE_df.drop(['level_0'], axis = 1)
print("SE:", len(SE_df))

#combined
df_T = (SA_df, SB_df, SC_df, SD_df, SE_df)

combined_MD_loc_df = pd.concat(df_T).reset_index()

print(combined_MD_loc_df)
print("combined:", len(combined_MD_loc_df))

In [ ]:
Antartica_poly = unary_union(gpd.read_file("./Quantarctica3/Basemap/ADD_Coastline_low_res_polygon.shp").to_crs('EPSG:3031').geometry)

#single geom for Antartica
Antarctica = Antartica_poly


#single geom for the coastline
coastline = unary_union(gpd.read_file("./Quantarctica3/Basemap/ADD_Coastline_high_res_line_Sliced.shp").to_crs('EPSG:3031').geometry)

x = combined_MD_loc_df['REMA_x']
y = combined_MD_loc_df['REMA_y']

data = list(zip(x,y))
Fake_df = pd.DataFrame(data, columns=['Longitude','Latitude'])


geometry = []
Dist_coast = []
for i in range(len(combined_MD_loc_df)):
    x = Fake_df.at[i,'Longitude']
    y = Fake_df.at[i,'Latitude']
    point = Point(x,y)
    P_xy = {"geometry": point}
    geometry.append(point)
    P_xy['dist_to_coastline_km'] = (P_xy['geometry'].distance(coastline))/1000
    #print(P_xy['dist_to_coastline_meters'])
    Dist_coast.append(P_xy['dist_to_coastline_km'])
    
data = list(zip(geometry,Dist_coast))
MD_dist_df = pd.DataFrame(data, columns=['geometry','Distance from Coastline (km)'])
#print(MD_dist_df)
#print(len(MD_dist_df))

MD_dist_df_km = MD_dist_df['Distance from Coastline (km)']


gdf = gpd.GeoDataFrame.from_records(MD_dist_df)
#print(gdf)

gdf1 = gpd.GeoDataFrame.from_records([{"geometry":coastline}])
print(gdf1)

ax = gpd.GeoDataFrame.from_records([{"geometry":Antartica_poly}]).plot(figsize=(18,18), color='gainsboro')
ax = gdf1.plot(ax=ax,column='geometry',color='black')
ax = gdf.plot(ax=ax, column='Distance from Coastline (km)', cmap='rainbow',legend=True,
              legend_kwds={'shrink': 0.7,'pad':0.01, 'orientation':'horizontal'})
fig = ax.figure
cb_ax = fig.axes[1] 
cb_ax.tick_params(labelsize=16)
cb_ax.set_xlabel('Distance from Coastline (km)', fontsize=20)
ax.axis('off')

plt.savefig('./indiv MD recordings Distance from Coastline.jpg', dpi=400)

plt.show()